In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
# MODEL = "gpt-3.5-turbo"
MODEL = "llama2"
# MODEL = "mixtral:8x7b"

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model= ChatOpenAI(api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else: 
    model=Ollama(model=MODEL)
    embeddings = OllamaEmbeddings()
model.invoke("What is Dueling DQN?")

'Dueling DQN (DDQN) is a type of deep reinforcement learning algorithm that combines two separate neural networks to learn both the policy and the value function of an agent simultaneously. The goal of DDQN is to learn a single neural network that can solve a given task by directly estimating both the policy and the value function.\n\nIn traditional Q-learning, the agent learns a policy that maps states to actions, and a value function that estimates the expected return for each state-action pair. However, this approach has some limitations. For example, it can be difficult to learn an accurate value function when the environment is complex or has a large state space. Additionally, learning both the policy and the value function separately can lead to suboptimal performance, as the policy may not take into account the information provided by the value function.\n\nDDQN addresses these limitations by using two separate neural networks to learn the policy and the value function simultane

In [3]:
# Used only for GPt Models
from langchain_core.output_parsers import StrOutputParser

parser= StrOutputParser()

chain = model | parser 
chain.invoke("Hey tell me a joke ")


"Sure! Here's a classic one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that brought a smile to your face! Do you want to hear another one?"

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("diffusion_models.pdf")
pages = loader.load_and_split()

pages

[Document(page_content='Understanding Diﬀusion Models: A Uniﬁed Perspective\nCalvin Luo\nGoogle Research, Brain Team\ncalvinluo@google.com\nAugust 26, 2022\nContents\nIntroduction: Generative Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nBackground: ELBO, VAE, and Hierarchical VAE . . . . . . . . . . . . . . . . . . . . . . . . 2\nEvidence Lower Bound . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2\nVariational Autoencoders . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\nHierarchical Variational Autoencoders . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nVariational Diﬀusion Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nLearning Diﬀusion Noise Parameters . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 14\nThree Equivalent Interpretations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 15\nScore-based Generative Mode

In [5]:
from langchain.prompts import PromptTemplate

template= """
Answer the Question based on the context below. 
If you cant answer the question, reply "I don't know!"

Context:{context}

Question:{question}

"""

prompt= PromptTemplate.from_template(template)
print(prompt.format(context="Here is a context", question="Here is a question"))




Answer the Question based on the context below. 
If you cant answer the question, reply "I don't know!"

Context:Here is a context

Question:Here is a question




In [8]:
prompt

PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the Question based on the context below. \nIf you cant answer the question, reply "I don\'t know!"\n\nContext:{context}\n\nQuestion:{question}\n\n')

In [6]:
chain = prompt | model | parser 

In [7]:
chain.invoke(
    {   
        "context": "I have studied in BVRIT",
        "question": "Where did i study?" 
    }
)

' Sure! Based on the context you provided, the answer to the question "Where did I study?" is likely "BVRIT".'

In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore= DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [33]:
retriever = vectorstore.as_retriever()

retriever.invoke("What are diffusion models?")# Control the number of documents using retriever top_key=2

[Document(page_content='Understanding Diﬀusion Models: A Uniﬁed Perspective\nCalvin Luo\nGoogle Research, Brain Team\ncalvinluo@google.com\nAugust 26, 2022\nContents\nIntroduction: Generative Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nBackground: ELBO, VAE, and Hierarchical VAE . . . . . . . . . . . . . . . . . . . . . . . . 2\nEvidence Lower Bound . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2\nVariational Autoencoders . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\nHierarchical Variational Autoencoders . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nVariational Diﬀusion Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nLearning Diﬀusion Noise Parameters . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 14\nThree Equivalent Interpretations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 15\nScore-based Generative Mode

In [34]:
from operator import itemgetter

chain = ({"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
        | prompt
        | model
        | parser 
)

chain.invoke({"question": "What are diffusion models?"})

'Diffusion models are a class of generative models that model the underlying distribution of a dataset by diffusing noise into the data. The goal is to learn a neural network that can predict the original ground truth image from an arbitrarily noisy version of it. In other words, diffusion models are a type of generative model that can generate new data samples that are similar to the training data but with added noise.\n\nThe basic idea behind diffusion models is to define a probability distribution over the pixels of an image, and then use a neural network to transform this distribution into the original ground truth image. The key insight is that the distribution over the pixels can be defined using a diffusion process, which is a continuous-time Markov process. By defining the distribution over the pixels in terms of a diffusion process, it is possible to model the underlying structure of the data in a more flexible and general way than traditional generative models.\n\nThere are s

In [35]:
questions = [
    "What are diffusion models? ",
    "How are diffusion models different from Text generation LLMs", 
    "What are diffusion noise parameters?"
]

for question in questions:
    print((f"Question: {question}"))
    print(f"Answer: {chain.invoke({'question': question})}") # use stream or batch to answer a batch of question (parallel processing)
    print()

Question: What are diffusion models? 
Answer: Diffusion models are a class of deep learning models that are designed to perform denoising, or removing noise from images, in a continuous and hierarchical manner. The basic idea behind diffusion models is to gradually corrupt an image over time by adding Gaussian noise, until the final output is identical to pure Gaussian noise. This process is depicted visually in Figure 3 of the paper.

The key insight of diffusion models is that the optimization of the evidence lower bound (ELBO) can be used as a proxy for maximizing the log-likelihood of the data. Specifically, the ELBO is equal to the sum of the KL divergence between the approximate posterior and the true posterior, plus the negative log probability of the data given the approximate posterior. By optimizing the ELBO, diffusion models can learn to denoise images in a way that is consistent with the true posterior distribution.

Diffusion models consist of two main components: a noise 

In [36]:
for s in chain.stream({"question": "How are diffusion models different from Text generation LLMs?"}):
    print(s, end="", flush=True)

Diffusion models and text generation LLMs are both machine learning models used in natural language processing, but they have some key differences:

1. Output structure: Diffusion models generate a continuous distribution over the input space, whereas text generation LLMs output a discrete sequence of words or tokens.
2. Training objective: Diffusion models are typically trained using a denoising autoencoder-like objective, where the goal is to reconstruct the original input from a noisy version. Text generation LLMs, on the other hand, are often trained using a masked language modeling objective, where the goal is to predict the missing word or token in a sequence given the context.
3. Input representation: Diffusion models typically represent the input as a continuous vector, whereas text generation LLMs may use a discrete representation such as a bag-of-words.
4. Generative capacity: Text generation LLMs are generally considered more powerful generators of coherent and fluent text c

In [1]:
ab= [{'page_number': 1, 'content': 'Hey this is a test PDF       '}, {'page_number': 2, 'content': 'Here is page 1     '}, {'page_number': 3, 'content': 'Here is page 2     '}, {'page_number': 4, 'content': 'Here is page 3  '}]
type(ab)


list

In [5]:
from langchain_core.documents.base import Document
ab = [{'page_number': 1, 'content': 'Hey this is a test PDF       '}, 
      {'page_number': 2, 'content': 'Here is page 1     '}, 
      {'page_number': 3, 'content': 'Here is page 2     '}, 
      {'page_number': 4, 'content': 'Here is page 3  '}]

documents = []

for item in ab:
    page_number = item['page_number']
    page_content = item['content']  # Rename 'content' to 'page_content'
    document = Document(page_content)
    documents.append(document)

In [10]:
documents

[Document(page_content='Hey this is a test PDF       '),
 Document(page_content='Here is page 1     '),
 Document(page_content='Here is page 2     '),
 Document(page_content='Here is page 3  ')]

In [15]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore= DocArrayInMemorySearch.from_documents(documents, embedding=embeddings)

In [16]:
vectorstore